### 查看GPU

In [1]:
!nvidia-smi #查看 GPU 的使用情況

Thu Apr 18 16:22:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [ ]:
!pip install transformers==4.29.0 -U #安裝 Transformers 套件
!pip install nlp -U #安裝 nlp 套件
!pip install torch==2.0.0 -U #安裝 PyTorch 的特定版本
!pip install transformers accelerate #安裝 Transformers 套件

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 3.4 MB/s eta 0:00:00
    

### 載入套件

In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments # Transformers 套件中引入了幾個重要的類別
from nlp import load_dataset, Dataset #引入了 load_dataset 函式和 Dataset 類
import nlp #引入了整個 nlp
import torch #深度學習
import random #生成隨機數。
import pandas as pd #深度學習
from sklearn.metrics import accuracy_score, precision_recall_fscore_support #計算分類任務的準確率 用於計算分類任務的精確率、召回率、F1分數和支持度

### 抓取訓練資料

In [4]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv #下載文件

--2024-04-18 16:38:50--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-18 16:38:51--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.04s   

2024-04-18 16:38:51 (10.1 MB/s) - ‘train.csv’ saved [388895/388895]



### 讀取訓練資料集

In [5]:
df = pd.read_csv('train.csv') #讀取數據
dataset = Dataset.from_pandas(df) #轉換為庫中的數據集

In [ ]:
df.sample(10)

,ID,Name,Description,label
345,345,素果青汁,「2週間前59.2公斤…現在46.8公斤…越吃越瘦的方法…脂肪分解瘦身法…世界上唯一取得瘦身...,1
19,19,纖先暢輕盈美麗勁爆價專案,「挑戰10天鑑賞期降5-8公斤...李芳雯口述：...減肥真的很簡單，不知不覺就瘦了！一個月...,1
80,80,長庚研發緩釋B群能量滿分專案,「長庚醫學團隊研發，新陳代謝科、減重門診、家醫科指定處方...女試用者竹庭口述：懷孕胖28公...,1
555,555,威瑪舒培新B群速崩緩釋長效組,…經實驗證實：…30分鐘燃燒脂肪…改善肥胖症狀…緩釋技術促進代謝燃燒…減重的秘密武器：補充維...,1
664,664,愛尚它?輔酵素Q10粉末,愛尚它?輔酵素Q10(CoQ10)粉末含有輔酵素Q10，維生素，以及脂肪分解酵素，有助於健康...,0
280,280,日華好物硒元素天然巴西堅果,「癌症醫生：癌症是可提前預防的…每日補充足夠硒元素，離癌症遠一步…健腦…增強記憶、防癌…美國...,1
78,78,長庚團隊舒暢益生菌健康組,「長庚臨床實驗結果九種複合益生菌預防大腸癌、便秘、消化不良、痔瘡、口臭...『具分解蛋白質、...,1
87,87,寒天,「蹲也蹲不下，站也站不起...關節」「即期品特惠買一送一喜康瑞-感謝寒天(180錠/瓶)」「...,1
244,244,BELTA白高顆膠囊,「瘦了6.1公斤…瘦了6公斤…酵素=減肥…瘦身體…瘦身輔助…瘦身…重啟身體機能」「想認真節食...,1
38,38,Tulloch澳洲花粉,「抗菌及降低膽固醇等功效..消除疲勞…保護肌膚，幫助細胞再生..治療皮膚過敏…對於肺部組織有...,1


### 載入原始模型

In [6]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')#創建了一個新的模型實例

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')#創建了一個新的分詞器

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 設定執行參數

In [8]:
RANDOM_SEED = 5 #設定了隨機種子
MAX_LEN = 512 #設定了輸入序列的最大長度
EPOCHS=5 #這個參數設定了訓練過程中的迭代次數
BATCH_SIZE = 8 #這個參數設定了每個訓練批次中包含的樣本數量

### 把資料集分成訓練、測試、驗證

In [9]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)#混洗操作，即將數據集中的樣本打亂順序
split_ds = shuffled_ds.train_test_split(test_size=0.2)#分割為訓練集和測試集指定了測試集的比例為20%
train_dataset = split_ds['train']#訓練集存儲在train_dataset變數中
test_val_dataset = split_ds['test']#測試集存儲在test_val_dataset變數中
split_tv = test_val_dataset.train_test_split(test_size=0.5)#進行分割 驗證集的比例為50%
test_dataset = split_tv['train']#測試集存儲在test_dataset變數中
val_dataset = split_tv['test']#驗證集存儲在val_dataset變數中

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [10]:
train_dataset[2] #獲取訓練集中索引為2的樣本

{'Description': '「唯一榮獲藍光保護專利的葉黃素品牌...本產品根據美國視覺醫學基礎研究學會ARVO發表的大型臨床研究AREDS-2文獻報告的建議配方」',
 'ID': 153,
 'Name': '美國Kemin開明植物膠囊（專利小分子葉黃素）（90顆/罐）',
 'label': 1}

In [11]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")#將給定的文本進行分詞，即將文本分解為其組成的詞或子詞

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [12]:
def tokenize(batch): #它對訓練集、測試集和驗證集進行了分詞處理，並將分詞後的結果轉換成PyTorch張量的格式
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [14]:
trainer.train()#訓練模型
trainer.evaluate()#評估模型

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.0006062323809601367,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 2.3757,
 'eval_samples_per_second': 39.566,
 'eval_steps_per_second': 5.051,
 'epoch': 5.0}

### 找一個範例來測試

In [16]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])#輸出數據集中的特定樣本的標籤和描述信息
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 設定測試文字

In [17]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'

MAX_LEN = 512 #輸入到預訓練的BERT模型中進行預測或特徵提取

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [18]:
encoded_review

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 進行測試

In [19]:
test_model = model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['合格', '不合格']

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')#根據預測的索引從類名列表中找到相應的類別名稱，並輸出最終的預測結果

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
